# Machine Intelligence with Deep Learning
## Importance batching for improved training of neural networks
---

In [1]:
import timeit

In [2]:
from models.resnet import ResNet18
from utils.data_utils import DataLoader
from utils.logging_utils import *

import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

import os
import pandas as pd

from datetime import datetime
today = datetime.today().strftime('%Y%m%d')

In [3]:
SEEDS = [10, 42, 4] # don't change!
STRATEGIES = ['freeze', 'shuffle', 'homogeneous', 'heterogeneous'] # can be changed

In [4]:
### Training
def train(epoch, optimizer, criterion, seed, dataloader, strategy):
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                                           random_state=seed, use_train=True)):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    train_acc = 100.*correct/total
    train_loss /= total
    return train_acc, train_loss

### Testing
def test(epoch, best_acc, seed, dataloader, strategy):
    net.eval()
    test_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(dataloader.yield_batches(strategy, \
                                                                               random_state=seed, use_train=False)):
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)

            test_loss += loss.item()
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()

    # Save checkpoint.
    test_acc = 100.*correct/total
    test_loss /= total
    if test_acc > best_acc:
        best_acc = test_acc
        net.save(best_acc, epoch, seed, strategy)
    return test_acc, test_loss

In [5]:
resume = False
given_date = '20191113'
device = 'cuda' if torch.cuda.is_available() else 'cpu'
if device == 'cuda':
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
### task: classification of the following classes
classes = ['plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

    
### hyperparameters
test_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch
num_epochs = 150 # number of iterations the model gets trained
learning_rates = { # learning rate is reset after specific epochs
    '1': 0.1, # 50 epochs
    '50': 0.01, # 40 epochs
    '90': 0.005, # 30 epochs
    '120': 0.001 # 30 epochs
}
momentum = 0.9
weight_decay = 5e-4

print("Begin training.")
start = timeit.default_timer()
#Logging header
length_table = 90
log_separating_line(length_table)
log_header_line("Seeds: {}".format(SEEDS), length_table)
log_header_line("Strategies: {}".format(STRATEGIES), length_table)
log_header_line("-> Resulting number of iterations: {}".format(len(SEEDS) * len(STRATEGIES)), length_table)
log_header_line("Number of iterations: {}".format(num_epochs), length_table)
log_header_line("Learning rates: {}".format(learning_rates), length_table)
log_header_line("Resuming from checkpoint: {}".format(True if resume else False), length_table)
log_separating_line(length_table)

rows = []
for seed in SEEDS:
    for strategy in STRATEGIES:
                
        np.random.seed(seed)
        torch.manual_seed(seed)
        if device == 'cuda':
            torch.cuda.manual_seed_all(seed)
             
        ### Model
        net = ResNet18()
        net = net.to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = optim.SGD(net.parameters(), lr=learning_rates['1'], momentum=momentum, weight_decay=weight_decay)
        
        ### load the data
        # if needed, specify batch sizes and shuffle settings
        dataloader = DataLoader()
        dataloader.download_cifar()
        print()
        log_separating_line(length_table)
        log_position_header(seed, strategy, length_table)
        log_separating_line(length_table)

        if resume:
            assert os.path.isdir('serialized'), 'Error: no serialized directory found!'
            ckpt = torch.load('./serialized/{}/{}_ckpt_{}.pth'.format(given_date, strategy, seed))
            test_acc, start_epoch, net = net.load(ckpt)

        for epoch in range(start_epoch, start_epoch+num_epochs):
            # reset learning rate at specific epochs
            if str(epoch+1) in learning_rates.keys():
                for param_group in optimizer.param_groups:
                    param_group['lr'] = learning_rates[str(epoch+1)]
                    
            train_acc, train_loss = train(epoch, optimizer, criterion, seed, dataloader, strategy)
            test_acc, test_loss = test(epoch, test_acc, seed, dataloader, strategy)
            log_position_line(epoch + 1, num_epochs, train_acc, test_acc, train_loss, test_loss, length_table)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': True,
                'strategy': strategy,
                'accuracy': train_acc,
                'loss': train_loss
            }
            rows.append(row)
            row = {
                'epoch': epoch + 1,
                'seed': seed,
                'train': False,
                'strategy': strategy,
                'accuracy': test_acc,
                'loss': test_loss
            }
            rows.append(row)
        log_separating_line(length_table)
            
stop = timeit.default_timer()
time_needed = stop - start
hrs = int(time_needed / 3600)
mins = int((time_needed / 60) % 60)
secs = int(time_needed % 60)
print()
print("Finished training. Time needed: {} hrs {} mins {} secs".format(hrs, mins, secs))

logging_df = pd.DataFrame(rows, columns=['epoch', 'seed', 'train', 'strategy', 'accuracy', 'loss'])   
training_logs_dir = 'evaluation_logs'
logging_df.to_csv('{}.txt'.format(os.path.join(training_logs_dir, today)), sep='\t', index=False)

Begin training.
+----------------------------------------------------------------------------------------+
| Seeds: [10, 42, 4]                                                                     |
| Strategies: ['freeze', 'shuffle', 'homogeneous', 'heterogeneous']                      |
| -> Resulting number of iterations: 12                                                  |
| Number of iterations: 150                                                              |
| Learning rates: {'1': 0.1, '50': 0.01, '90': 0.005, '120': 0.001}                      |
| Resuming from checkpoint: False                                                        |
+----------------------------------------------------------------------------------------+
Files already downloaded and verified
Files already downloaded and verified

+----------------------------------------------------------------------------------------+
| Seed: 10      Strategy: freeze                                                        

| [079/150]:    100.0           87.29           003e-05      0.00709                     |
| [080/150]:    100.0           87.19           003e-05      0.00709                     |
| [081/150]:    100.0           87.13           003e-05      00.0071                     |
| [082/150]:    100.0           87.16           003e-05      0.00712                     |
| [083/150]:    100.0           87.15           003e-05      0.00714                     |
| [084/150]:    100.0           87.12           003e-05      0.00716                     |
| [085/150]:    100.0           87.07           003e-05      00.0072                     |
| [086/150]:    100.0           87.05           003e-05      0.00723                     |
| [087/150]:    100.0           87.06           003e-05      0.00727                     |
| [088/150]:    100.0           087.0           003e-05      00.0073                     |
| [089/150]:    100.0           87.02           003e-05      0.00735                     |

| [014/150]:    83.84           74.47           0.00736      0.01335                     |
| [015/150]:    84.19           76.79           00.0072      0.01148                     |
| [016/150]:    84.54           71.15           0.00705      00.0149                     |
| [017/150]:    84.83           77.86           0.00689      00.0106                     |
| [018/150]:    85.14           75.14           0.00685      0.01252                     |
| [019/150]:    85.26           79.57           0.00677      0.01044                     |
| [020/150]:    85.22           076.6           00.0067      0.01164                     |
| [021/150]:    085.6           077.0           0.00663      0.01158                     |
| [022/150]:    85.57           73.93           00.0066      0.01286                     |
| [023/150]:    85.57           69.74           0.00658      0.01862                     |
| [024/150]:    85.81           074.8           0.00648      0.01236                     |

| [105/150]:    100.0           89.22           003e-05      0.00604                     |
| [106/150]:    100.0           89.33           003e-05      0.00603                     |
| [107/150]:    100.0           89.35           003e-05      0.00602                     |
| [108/150]:    100.0           89.38           004e-05      0.00615                     |
| [109/150]:    100.0           89.45           003e-05      0.00612                     |
| [110/150]:    100.0           89.41           003e-05      0.00596                     |
| [111/150]:    100.0           89.54           003e-05      0.00597                     |
| [112/150]:    100.0           089.3           004e-05      0.00613                     |
| [113/150]:    100.0           89.26           003e-05      0.00603                     |
| [114/150]:    100.0           089.4           003e-05      0.00594                     |
| [115/150]:    100.0           89.47           003e-05      0.00597                     |

| [040/150]:    81.42           010.0           0.02616      0.20365                     |
| [041/150]:    81.68           010.0           0.02614      0.20318                     |
| [042/150]:    81.55           010.0           0.02608      0.20405                     |
| [043/150]:    081.3           010.0           0.02629      00.2002                     |
| [044/150]:    81.55           010.0           0.02592      0.20456                     |
| [045/150]:    81.55           010.0           0.02608      0.20564                     |
| [046/150]:    81.81           010.0           0.02611      0.20629                     |
| [047/150]:    81.55           010.0           0.02621      0.20458                     |
| [048/150]:    81.68           010.0           0.02608      0.20574                     |
| [049/150]:    81.68           010.0           0.02625      0.20307                     |
| [050/150]:    38.54           010.0           0.04577      0.05694                     |

...

| [040/150]:    86.45           78.02           0.00623      0.01147                     |
| [041/150]:    86.53           77.13           0.00613      00.0109                     |
| [042/150]:    86.37           75.53           0.00625      0.01266                     |
| [043/150]:    86.59           70.78           0.00614      0.01552                     |
| [044/150]:    86.54           080.3           0.00614      0.00978                     |
| [045/150]:    86.45           76.79           0.00623      0.01177                     |
| [046/150]:    86.67           74.94           0.00609      0.01326                     |
| [047/150]:    86.64           77.79           0.00612      0.01094                     |
| [048/150]:    86.76           080.9           00.0061      0.00907                     |
| [049/150]:    86.61           73.86           0.00611      00.0135                     |
| [050/150]:    95.76           87.53           0.00209      0.00629                     |

| [131/150]:    100.0           89.35           003e-05      0.00599                     |
| [132/150]:    100.0           89.26           003e-05      0.00599                     |
| [133/150]:    100.0           89.25           003e-05      0.00599                     |
| [134/150]:    100.0           089.5           003e-05      0.00599                     |
| [135/150]:    100.0           89.32           003e-05      0.00606                     |
| [136/150]:    100.0           89.34           003e-05      000.006                     |
| [137/150]:    100.0           89.13           003e-05      0.00599                     |
| [138/150]:    100.0           89.41           003e-05      0.00599                     |
| [139/150]:    100.0           89.44           003e-05      0.00605                     |
| [140/150]:    100.0           89.36           003e-05      0.00605                     |
| [141/150]:    100.0           089.4           003e-05      0.00598                     |

| [066/150]:    50.83           010.0           0.02762      0.05967                     |
| [067/150]:    55.06           010.0           0.02557      0.05826                     |
| [068/150]:    55.44           010.0           0.02532      0.05918                     |
| [069/150]:    51.47           010.0           0.02697      0.05777                     |
| [070/150]:    50.58           010.0           0.02777      0.05831                     |
| [071/150]:    55.06           010.0           0.02573      0.05856                     |
| [072/150]:    52.24           010.0           0.02636      0.05821                     |
| [073/150]:    50.83           010.0           0.02768      0.05767                     |
| [074/150]:    050.7           010.0           00.0278      0.05847                     |
| [075/150]:    51.34           010.0           0.02713      0.05876                     |
| [076/150]:    50.58           010.0           0.02785      0.05848                     |

| [001/150]:    36.19           46.55           0.02771      0.02292                     |
| [002/150]:    52.63           55.51           0.02041      0.02055                     |
| [003/150]:    63.76           60.82           0.01597      0.01843                     |
| [004/150]:    72.25           70.25           0.01247      00.0136                     |
| [005/150]:    76.75           71.84           0.01052      0.01308                     |
| [006/150]:    79.02           074.7           0.00946      0.01194                     |
| [007/150]:    81.04           75.34           00.0087      0.01216                     |
| [008/150]:    82.23           78.04           0.00813      0.01038                     |
| [009/150]:    82.91           75.84           0.00777      0.01141                     |
| [010/150]:    83.68           074.7           0.00741      0.01199                     |
| [011/150]:    84.29           77.08           0.00709      00.0113                     |

| [092/150]:    100.0           88.52           002e-05      0.00705                     |
| [093/150]:    100.0           88.66           001e-05      0.00697                     |
| [094/150]:    100.0           88.64           002e-05      0.00691                     |
| [095/150]:    100.0           88.76           002e-05      0.00682                     |
| [096/150]:    100.0           88.73           002e-05      0.00668                     |
| [097/150]:    100.0           88.71           002e-05      0.00668                     |
| [098/150]:    100.0           88.77           002e-05      0.00657                     |
| [099/150]:    100.0           88.74           002e-05      0.00653                     |
| [100/150]:    100.0           88.79           002e-05      00.0065                     |
| [101/150]:    100.0           88.89           002e-05      0.00657                     |
| [102/150]:    100.0           88.94           002e-05      0.00645                     |